In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
import lightgbm as lgb
from catboost import CatBoostClassifier
from hillclimbers import climb_hill, partial
import warnings


/opt/miniconda3/envs/kaggleEnv/lib/python3.12/site-packages/hillclimbers/__init__.py:44: SyntaxWarning: invalid escape sequence '\ '
  hill_icon = f"{BLUE_TXT}   /\\  \n  /__\  hillclimbers{RESET_TXT}{BOLD_TXT} \n /    \\\n/______\\ \n{RESET_TXT}"


In [5]:

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Data Loading
train = pd.read_csv('../dataset/train.csv')
test = pd.read_csv('../dataset/test.csv')
original = pd.read_csv('../dataset/original.csv')
submission = pd.read_csv('../dataset/sample_submission.csv')

train = pd.concat([train, original], ignore_index=True)
train['id'] = np.arange(len(train))


In [6]:


target = "loan_status"

# Combine train and test for consistent preprocessing
combined = pd.concat([train.drop(columns=[target]), test], ignore_index=True)

# Data Preprocessing
# Fill missing values
combined['person_emp_length'].fillna(combined['person_emp_length'].median(), inplace=True)
combined['loan_int_rate'].fillna(combined['loan_int_rate'].median(), inplace=True)

# Encode categorical variables
categorical_cols = ['person_home_ownership', 'loan_intent', 'loan_grade', 'cb_person_default_on_file']
for col in categorical_cols:
    le = LabelEncoder()
    combined[col] = le.fit_transform(combined[col])

# Split combined data back into train and test
X = combined.iloc[:len(train), :].drop(columns=['id'])
X_test = combined.iloc[len(train):, :].drop(columns=['id'])
y = train[target]

# Cross-validation strategy
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Initialize dictionaries to store out-of-fold and test predictions
oof_preds = {}
test_preds = {}


In [8]:

# Define evaluation metric
eval_metric = partial(roc_auc_score)

# XGBoost parameters
xgb_params = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'use_label_encoder': False,
    'random_state': 42,
    'n_estimators': 1000,
    'learning_rate': 0.05,
    'max_depth': 6,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    "early_stopping_rounds":50,
}

# LightGBM parameters
lgb_params = {
    'objective': 'binary',
    'metric': 'auc',
    'random_state': 42,
    'n_estimators': 1000,
    'learning_rate': 0.05,
    'max_depth': 6,
    'num_leaves': 31,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
}

# CatBoost parameters
cat_params = {
    'loss_function': 'Logloss',
    'eval_metric': 'AUC',
    'random_seed': 42,
    'iterations': 1000,
    'learning_rate': 0.05,
    'depth': 6,
    'verbose': False,
}

models = {
    'XGB': (XGBClassifier, xgb_params),
    'LGB': (lgb.LGBMClassifier, lgb_params),
    'CAT': (CatBoostClassifier, cat_params),
}

# Perform cross-validation for each model
for model_name, (ModelClass, params) in models.items():
    print(f"Training {model_name}...")
    oof_pred = np.zeros(X.shape[0])
    test_pred = np.zeros(X_test.shape[0])

    for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
        X_tr, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_tr, y_val = y.iloc[train_idx], y.iloc[val_idx]

        if model_name == 'CAT':
            model = ModelClass(**params)
            model.fit(
                X_tr, y_tr,
                eval_set=(X_val, y_val),
                use_best_model=True,
                early_stopping_rounds=50,
            )
        elif model_name == 'LGB':
            model = ModelClass(**params)
            model.fit(
                X_tr, y_tr,
                eval_set=[(X_val, y_val)],
                eval_names=['valid'],
                callbacks=[
                    lgb.early_stopping(stopping_rounds=50),
                    lgb.log_evaluation(period=0)
                ],
            )
        else:  # XGBoost
            model = ModelClass(**params)
            model.fit(
                X_tr, y_tr,
                eval_set=[(X_val, y_val)],
                
                verbose=False
            )

        oof_pred[val_idx] = model.predict_proba(X_val)[:, 1]
        test_pred += model.predict_proba(X_test)[:, 1] / n_splits

    oof_preds[model_name] = oof_pred
    test_preds[model_name] = test_pred

# Convert predictions to DataFrames
oof_pred_df = pd.DataFrame(oof_preds)
test_pred_df = pd.DataFrame(test_preds)

# Prepare train DataFrame with target
train_df = pd.DataFrame({
    'loan_status': y
})

# Use the provided climb_hill function (make sure it's defined in your environment)
blended_test_preds = climb_hill(
    train=train_df,
    oof_pred_df=oof_pred_df,
    test_pred_df=test_pred_df,
    target='loan_status',
    objective='maximize',
    eval_metric=eval_metric,
    negative_weights=False,
    precision=0.01,
    plot_hill=True,
    plot_hist=True,
    return_oof_preds=False
)


Training XGB...
Training LGB...
[LightGBM] [Info] Number of positive: 12366, number of negative: 60614
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000652 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 885
[LightGBM] [Info] Number of data points in the train set: 72980, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.169444 -> initscore=-1.589575
[LightGBM] [Info] Start training from score -1.589575
Training until validation scores don't improve for 50 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positi

In [9]:

# Prepare submission
submission[target] = blended_test_preds
submission.to_csv('submission.csv', index=False)
print("\nSubmission file created: 'submission.csv'")



Submission file created: 'submission.csv'
